In [1]:
import torch
import numpy as np
from utils import get_score#, get_loss_weight
result_dir='/home/dyj/'
sigmoid = torch.sigmoid

In [2]:
label = torch.load('/home/dyj/label.pt')

In [ ]:
# cnn1 = torch.load('/home/dyj/TextCNN1_2017-07-27#10:15:20_res.pt')
# cnn1_loss_weight = torch.load(result_dir+'TextCNN1_loss_weight.pt')

In [ ]:
rnn1 = torch.load('/home/dyj/RNN1_2017-07-27#10:48:05_res.pt')
rnn1_loss_weight = torch.load(result_dir+'RNN1_loss_weight.pt')

In [ ]:
# rcnn1 = torch.load(result_dir + 'RCNN1_2017-07-27#11:01:07_res.pt')
# rcnn1_loss_weight = torch.load('/home/dyj/RCNN1_loss_weight.pt')

In [ ]:
rcnncha = torch.load('/home/dyj/RCNNcha_2017-07-27#16:19:23_res.pt')
rcnncha_loss_weight = torch.load(result_dir+'RCNNcha_loss_weight.pt')

In [ ]:
fasttext1_char = torch.load('snapshots/FastText/layer_1_cal_res_char.pt')
fasttext1_loss_weight = torch.load('snapshots/FastText/layer_2_loss_weight_char.pt')

In [ ]:
# fasttext4 = torch.load(result_dir + 'FastText4_2017-07-28#15:14:47_res.pt')
# fasttext4_loss_weight = torch.load('/home/dyj/FastText4_loss_weight.pt')

In [ ]:
fasttext10 = torch.load('/home/dyj/FastText10_res.pt')
fasttext10_loss_weight = torch.load('/home/dyj/FastText10_loss_weight.pt')

In [3]:
rnn10_cnn7 = torch.load('snapshots/TextCNN/layer_17_cal_res_3.pt')
rnn10_cnn7_loss_weight = torch.load('snapshots/TextCNN/layer_18_loss_weight_3.pt')

In [ ]:
cnn4 = torch.load('snapshots/TextCNN/layer_4_cal_res_3.pt')
cnn4_loss_weight = torch.load('snapshots/TextCNN/layer_5_loss_weight_3.pt')

In [5]:
def get_loss_weight(logit, label):
    class_num = logit.size(1)
    predict_label_list = [list(ii) for ii in logit.topk(5, 1)[1]]
    marked_label_list = [list(np.where(ii.numpy()==1)[0]) for ii in label]
    sample_per_class = torch.zeros(class_num)
    error_per_class = torch.zeros(class_num)
    for predict_labels, marked_labels in zip(predict_label_list, marked_label_list):
        for true_label in marked_labels:
            sample_per_class[true_label] += 1
            if true_label not in predict_labels:
                error_per_class[true_label] += 1
    return error_per_class / sample_per_class

In [6]:
cnn4_top1 = torch.load('snapshots/TextCNN/layer_4_cal_res_top1.pt')
cnn4_top1_loss_weight = get_loss_weight(cnn4_top1, label)

In [ ]:
#torch.save(cnn3_loss_weight, 'results/TextCNN3_top1_loss_weight_5.pt')

In [ ]:
# logit = sigmoid(rnn1) * torch.sqrt(1-rnn1_loss_weight+rnn1_loss_weight.mean()).expand_as(rnn1) * 0.03 + \
#         sigmoid(rcnncha) * torch.sqrt(1-rcnncha_loss_weight+rcnncha_loss_weight.mean()).expand_as(rcnncha) * 0.01 + \
#         sigmoid(fasttext1_char) * torch.sqrt(1-fasttext1_loss_weight+fasttext1_loss_weight.mean()).expand_as(fasttext1_char) * 0.03 + \
#         sigmoid(fasttext10/10) * torch.sqrt(1-fasttext10_loss_weight+fasttext10_loss_weight.mean()).expand_as(fasttext10) * 0.05 + \
#         sigmoid(rnn10_cnn7/17) * torch.sqrt(1-rnn10_cnn7_loss_weight+rnn10_cnn7_loss_weight.mean()).expand_as(rnn10_cnn7) * 0.7 + \
#         sigmoid(cnn3/3) * torch.sqrt(1-cnn3_loss_weight+cnn3_loss_weight.mean()).expand_as(cnn3) * 0.2 + \
#         sigmoid(cnn4/4) * torch.sqrt(1-cnn4_loss_weight+cnn4_loss_weight.mean()).expand_as(cnn4) * 0.2

In [9]:
logit = sigmoid(rnn10_cnn7/17) * torch.sqrt(1-rnn10_cnn7_loss_weight+rnn10_cnn7_loss_weight.mean()).expand_as(rnn10_cnn7) * 0.7 + \
        sigmoid(cnn4_top1/4) * torch.sqrt(1-cnn4_top1_loss_weight+cnn4_top1_loss_weight.mean()).expand_as(cnn4_top1) * 0.2

In [10]:
get_score(logit, label)

(1.4838693534530871, 0.6062642592861598, 0.43041121819464268)

In [ ]:
get_score(logit, label)

In [ ]:
%cd utils
from resmat import search_model_weight

In [ ]:
models = [
         sigmoid(rnn1) * torch.sqrt(1-rnn1_loss_weight+rnn1_loss_weight.mean()).expand_as(rnn1),\
         sigmoid(rcnncha) * torch.sqrt(1-rcnncha_loss_weight+rcnncha_loss_weight.mean()).expand_as(rcnncha),\
         sigmoid(fasttext1_char) * torch.sqrt(1-fasttext1_loss_weight+fasttext1_loss_weight.mean()).expand_as(fasttext1_char),\
         sigmoid(fasttext10/10) * torch.sqrt(1-fasttext10_loss_weight+fasttext10_loss_weight.mean()).expand_as(fasttext10),\
         sigmoid(rnn10_cnn7/17) * torch.sqrt(1-rnn10_cnn7_loss_weight+rnn10_cnn7_loss_weight.mean()).expand_as(rnn10_cnn7),\
         sigmoid(cnn3/3) * torch.sqrt(1-cnn3_loss_weight+cnn3_loss_weight.mean()).expand_as(cnn3),\
         sigmoid(cnn4/4) * torch.sqrt(1-cnn4_loss_weight+cnn4_loss_weight.mean()).expand_as(cnn4)
         ]
logit = torch.stack(models, 2)

In [ ]:
init_weight = torch.Tensor([0.0293, 0.0127, 0.0281, 0.0466, 0.4903, 0.1633, 0.1254])

In [ ]:
cur_weight = torch.load('../snapshots/Stack/model7_weight.pt')
cur_weight

In [ ]:
search_model_weight(logit, label, cur_weight, 10, 0.0001, '../snapshots/Stack/model7_weight.pt')

In [ ]:
layer_11 = torch.load('snapshots/TextCNN/layer_11_cal_res_char.pt')
layer_12 = torch.load('snapshots/TextCNN/layer_12_cal_res_char.pt')

In [ ]:
layer_12_weighted = (layer_11 / 11 * 5.5 + (layer_12-layer_11)) / 6.5 * 12
layer_12_loss_weight = get_loss_weight(layer_12_weighted, label)

In [ ]:
print get_score(layer_12_weighted, label)
get_score(sigmoid(layer_12_weighted/12) * torch.sqrt(1-layer_12_loss_weight+layer_12_loss_weight.mean()).expand_as(layer_12_weighted), label)

In [ ]:
torch.save(layer_12_weighted, 'snapshots/TextCNN/layer_12_weighted_cal_res_char.pt')
torch.save(layer_12_loss_weight, 'snapshots/TextCNN/layer_13_weighted_loss_weight_char.pt')

In [ ]:
rnn_cnn2_test = torch.load('results/RNN10_CNN2_test_res.pt')
cnn3_test = torch.load('results/CNN3_2017-08-07#12:41:45_test_res.pt')
cnn2_test = torch.load('results/CNN2_2017-08-07#12:53:50_test_res.pt')
torch.save(rnn_cnn2_test+cnn3_test+cnn2_test, '/home/dyj/RNN10_CNN7_test_res.pt')
torch.save(rnn_cnn2_test+cnn3_test+cnn2_test, 'results/RNN10_CNN7_test_res.pt')

In [ ]:
logit = sigmoid(cnn1) * torch.sqrt(1-cnn1_loss_weight+cnn1_loss_weight.mean()).expand_as(cnn1) * 0 + \
        sigmoid(rnn1) * torch.sqrt(1-rnn1_loss_weight+rnn1_loss_weight.mean()).expand_as(rnn1) * 0. + \
        sigmoid(rcnn1) * torch.sqrt(1-rcnn1_loss_weight+rcnn1_loss_weight.mean()).expand_as(rcnn1) * 0 + \
        sigmoid(rcnncha) * torch.sqrt(1-rcnncha_loss_weight+rcnncha_loss_weight.mean()).expand_as(rcnncha) * 0 + \
        sigmoid(fasttext1_char) * torch.sqrt(1-fasttext1_loss_weight+fasttext1_loss_weight.mean()).expand_as(fasttext1_char) * 0 + \
        sigmoid(fasttext4/4) * torch.sqrt(1-fasttext4_loss_weight+fasttext4_loss_weight.mean()).expand_as(fasttext4) * 0 + \
        sigmoid(fasttext10/10) * torch.sqrt(1-fasttext10_loss_weight+fasttext10_loss_weight.mean()).expand_as(fasttext10) * 0.0469 + \
        sigmoid(rnn10_cnn7/17) * torch.sqrt(1-rnn10_cnn7_loss_weight+rnn10_cnn7_loss_weight.mean()).expand_as(rnn10_cnn7) * 0.6604 + \
        sigmoid(cnn4/4) * torch.sqrt(1-cnn4_loss_weight+cnn4_loss_weight.mean()).expand_as(cnn4) * 0.1536

In [ ]:
get_loss_weight??
rnn10_cnn8_loss_weight_new = get_loss_weight(rnn10_cnn8, label)

In [ ]:
torch.save(rnn10_cnn8_loss_weight_new, 'snapshots/TextCNN/layer_19_loss_weight_top1.pt')

In [ ]:
layer_18_loss_weight_top3 = torch.load('snapshots/TextCNN/layer_18_loss_weight_top3.pt')
layer_19_loss_weight_top3 = torch.load('snapshots/TextCNN/layer_19_loss_weight_top3.pt')

In [ ]:
print layer_18_loss_weight_top3.mean()
layer_19_loss_weight_top3.mean()